In [1]:
from timeit import default_timer as time

start_time = time()

import sys
path2oti = '../../../build/'
sys.path.append(path2oti) # Add path to OTI library.

import pyoti.real   as r
import pyoti.sparse as oti 
import pyoti.core   as coti
import pyoti.fem    as fem 

import pyoti.static.onumm1n1    as dual
import pyoti.static.mdnum2      as md2
import pyoti.static.onumm1n10   as om1n10
import pyoti.static.onumm1n2    as om1n2

end_time = time()
print("Time to load pyoti: {0:.4f} seg.".format(end_time-start_time))


%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.sparse.linalg as spla
e  = oti.e
np = oti.np

import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')



Time to load pyoti: 3.7805 seg.


In [2]:
def cylinder(ri, ro, h ,he = 1.0, element_order = 1, quads = False, 
           quad_incomplete = 1, quad_linear = 1, structured = False, save=False):
    """
    PORPUSE: Define a thick walled cylinder section mesh.
    """
    #***************************************************************************************************
    import gmsh

    gmsh.initialize()
    # gmsh.fltk.initialize()

    # Lets create a simple square element:
    model = gmsh.model
    geo   = model.geo
    option= gmsh.option

    P1 = geo.addPoint( 0.0, 0.0, 0.0, he, 1)
    P2 = geo.addPoint( 0.0,  ri, 0.0, he, 2)
    P3 = geo.addPoint(  ri, 0.0, 0.0, he, 3)
    P4 = geo.addPoint( 0.0,  ro, 0.0, he, 4)
    P5 = geo.addPoint(  ro, 0.0, 0.0, he, 5)

    C1 = geo.addCircleArc(P2,P1,P3)
    C2 = geo.addCircleArc(P5,P1,P4)
    
    geo.remove([P1])
    
    L1 = geo.addLine(P4,P2) # Left
    L2 = geo.addLine(P3,P5) # bottom

    loop1 = geo.addCurveLoop([C1,L2,C2,L1])

    surface = geo.addPlaneSurface([loop1])

    nels_h = int(h/he)+1
    extrude_tags = geo.extrude([(2,surface)],0,0,1,[nels_h],[h], recombine=quads)

    model.addPhysicalGroup( 0, [P1],        100 )
    model.addPhysicalGroup( 0, [P2],        101 )
    model.addPhysicalGroup( 0, [P3],        102 )
    model.addPhysicalGroup( 0, [P4],        103 )
    model.addPhysicalGroup( 0, [P5],        104 )

    model.addPhysicalGroup( 1, [C1],        201 )
    model.addPhysicalGroup( 1, [C2],        202 )
    model.addPhysicalGroup( 1, [L1],        203 )
    model.addPhysicalGroup( 1, [L2],        204 )

    model.addPhysicalGroup( 2, [surface],   301 )
    model.addPhysicalGroup( 2, [extrude_tags[0][1]],   302 )
    model.addPhysicalGroup( 2, [extrude_tags[2][1]],   303 )
    model.addPhysicalGroup( 2, [extrude_tags[3][1]],   304 )
    model.addPhysicalGroup( 2, [extrude_tags[4][1]],   305 )
    model.addPhysicalGroup( 2, [extrude_tags[5][1]],   306 )
    
    
    model.addPhysicalGroup( 3, [extrude_tags[1][1]],   401 )
    
    model.setPhysicalName( 0, 100, "center"  )
    model.setPhysicalName( 0, 101, "ri_left"  )
    model.setPhysicalName( 0, 102, "ri_right" )
    model.setPhysicalName( 0, 103, "ro_left"  )
    model.setPhysicalName( 0, 104, "ro_right" )

    model.setPhysicalName( 1, 201, "ri"    )
    model.setPhysicalName( 1, 202, "ro"    )
    model.setPhysicalName( 1, 203, "left"  )
    model.setPhysicalName( 1, 204, "right" )

    model.setPhysicalName( 2, 301, "bottom_face")
    model.setPhysicalName( 2, 302, "top_face"   )
    model.setPhysicalName( 2, 303, "ri_face"    )
    model.setPhysicalName( 2, 305, "ro_face"    )
    model.setPhysicalName( 2, 304, "right_face"  )
    model.setPhysicalName( 2, 306, "left_face" )
    
    
    model.setPhysicalName( 3, 401, "domain"      )

    
    geo.synchronize()

    option.setNumber('Mesh.ElementOrder',element_order)


    # Recombine if quads are wanted.
    if quads:

        # Set body to recombine into quads.
        option.setNumber('Mesh.SecondOrderIncomplete',quad_incomplete)
        option.setNumber('Mesh.SecondOrderLinear',    quad_linear    )
        option.setNumber('Mesh.RecombineAll',         1)

    else:

        option.setNumber('Mesh.SecondOrderIncomplete',quad_incomplete)
        option.setNumber('Mesh.SecondOrderLinear',    quad_linear    )
        option.setNumber('Mesh.RecombineAll',         0)

    # end if 

    if structured:
        nels_L = int((ro-ri)/he)+1
        nels_C = int((ri*np.pi*2)/(4*he))+1
        model.mesh.setTransfiniteCurve(L1,nels_L)
        model.mesh.setTransfiniteCurve(L2,nels_L)
        model.mesh.setTransfiniteCurve(C1,nels_C)
        model.mesh.setTransfiniteCurve(C2,nels_C)
        model.mesh.setTransfiniteSurface(surface)#,"left",[P2,P3,P4])
#         model.mesh.setTransfiniteVolume(extrude_tags[1][1])
    # end if 

    model.mesh.generate(3)

    if save:
        gmsh.write("cylinder.msh")
    # end if 

    Th = fem.mesh.from_gmsh(gmsh)
#     Th = None

    gmsh.finalize()

    return Th

#-----------------------------------------------------------------------------------------------------

In [3]:
#*****************************************************************************************************
def solve_3d_linear_elasticity_prealloc(Th, E, nu, ri, Pi, ro, Po, stats=True):
    global times
    from timeit import default_timer as time
    
    start_time = time()
    
    mu = E/(2*(1+nu))
    
    c1 = E*nu/((1+nu)*(1-2*nu))
    c2 = 2*mu
    c3 = mu
    c4 = c1+c2
    
    ndim_analysis = 3
    els = Th.elements[3]

    fem.end_elements()
    
    nNodes = Th.x.shape[0]
    nDOF = 3*nNodes
    
    
    K = alg.lil_matrix((nDOF,nDOF))
    f = alg.zeros((nDOF,1))
    
    for i in unused_nodes:
        K[ i +        0, i +        0 ] = 1.0 # Removes "Unused" node.
        K[ i +   nNodes, i +   nNodes ] = 1.0 # Removes "Unused" node.
        K[ i + 2*nNodes, i + 2*nNodes ] = 1.0 # Removes "Unused" node.
    # 
    for j in range(els['types'].size):

        elem = fem.element[ els['types'][j] ]

        if not elem.is_allocated():

            elem.end()
            elem.allocate(intorder=2)
            elem.allocate_spatial(ndim_analysis,compute_Jinv = True)

            fh = alg.zeros( ( elem.nbasis, 1 ) )
            
            # Temps
            
            NxT = alg.zeros( ( elem.nbasis, 1 ), nip = elem.nip )
            NyT = alg.zeros( ( elem.nbasis, 1 ), nip = elem.nip )
            NzT = alg.zeros( ( elem.nbasis, 1 ), nip = elem.nip )
            
            NxNx = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            NxNy = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            NxNz = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            
            NyNx = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            NyNy = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            NyNz = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            
            NzNx = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            NzNy = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            NzNz = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )

            NN_tmp1 = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            NN_tmp2 = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            NN_tmp3 = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            NN_tmp4 = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            NN_tmp5 = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            
            tmp11 = alg.zeros( ( elem.nbasis, elem.nbasis ) )
            tmp12 = alg.zeros( ( elem.nbasis, elem.nbasis ) )
            tmp13 = alg.zeros( ( elem.nbasis, elem.nbasis ) )
            
            tmp21 = alg.zeros( ( elem.nbasis, elem.nbasis ) )
            tmp22 = alg.zeros( ( elem.nbasis, elem.nbasis ) )
            tmp23 = alg.zeros( ( elem.nbasis, elem.nbasis ) )
            
            tmp31 = alg.zeros( ( elem.nbasis, elem.nbasis ) )
            tmp32 = alg.zeros( ( elem.nbasis, elem.nbasis ) )
            tmp33 = alg.zeros( ( elem.nbasis, elem.nbasis ) )

        # end if 

        elm_nodes = els['indices'][j]
        num_elements = elm_nodes.shape[0]
        for i in range(elm_nodes.shape[0]):
            
            elems = elm_nodes[i,:]
            
            elem.set_coordinates(Th.x,Th.y,Th.z,elems)
            elem.compute_jacobian()

            Nx = elem.Nx
            Ny = elem.Ny
            Nz = elem.Nz
            N  = elem.N
            
            alg.transpose(Nx, out = NxT)
            alg.transpose(Ny, out = NyT)
            alg.transpose(Nz, out = NzT)
            
            alg.dot( NxT, Nx, out = NxNx )
            alg.dot( NxT, Ny, out = NxNy )
            alg.dot( NxT, Nz, out = NxNz )
            
            alg.dot( NyT, Nx, out = NyNx )
            alg.dot( NyT, Ny, out = NyNy )
            alg.dot( NyT, Nz, out = NyNz )
            
            alg.dot( NzT, Nx, out = NzNx )
            alg.dot( NzT, Ny, out = NzNy )
            alg.dot( NzT, Nz, out = NzNz )
            
        

            # problem = intV( c1*dx(vx)*dx(ux) + c1*dy(vy)*dx(ux) ) + \
            #           intV( c1*dx(vx)*dy(uy) + c1*dy(vy)*dy(uy) ) + \
            #           intV( c2*dx(vx)*dx(ux) + c2*dy(vy)*dy(uy) ) + \
            #           intV( c3*dy(vx)*dy(ux) + c3*dx(vy)*dy(ux) ) + \
            #           intV( c3*dy(vx)*dx(uy) + c3*dx(vy)*dx(uy) ) + \
            #           intS( 'in' , fxi*vx + fyi*vy ) + \
            #           intS( 'out', fxo*vx + fyo*vy ) + \
            #           on(   'left' , ux, 0.0) + \
            #           on(   'right', uy, 0.0)
            #  
            # c1*NxNx + c2*NxNx + c3*NyNy # vx, ux
            # c1*NxNy + c3*NyNx           # vx, uy
            # c1*NyNx + c3*NxNy           # vy, ux
            # c1*NyNy + c2*NyNy + c3*NxNx # vy, uy

            if alg is r:
                
                tmp11[:,:]=0.0
                tmp12[:,:]=0.0
                tmp13[:,:]=0.0
                
                tmp21[:,:]=0.0
                tmp22[:,:]=0.0
                tmp23[:,:]=0.0
                
                tmp31[:,:]=0.0
                tmp32[:,:]=0.0
                tmp33[:,:]=0.0
                
            else:
                tmp11.set(0)
                tmp12.set(0)
                tmp13.set(0)

                tmp21.set(0)
                tmp22.set(0)
                tmp23.set(0)

                tmp31.set(0)
                tmp32.set(0)
                tmp33.set(0)
            # end if 

            # alg.gauss_integrate( c4*NxNx + c3*NyNy , elem.dV,out=tmp11) # vx,ux            
            alg.mul(      c4,    NxNx, out=NN_tmp1 )
            alg.mul(      c3,    NyNy, out=NN_tmp2 )
            alg.sum( NN_tmp1, NN_tmp2, out=NN_tmp3 )
            alg.gauss_integrate( NN_tmp3, elem.dV, out = tmp11 ) # vx,ux
            
            # alg.gauss_integrate( c1*NxNy + c3*NyNx , elem.dV,out=tmp12) # vx,uy
            alg.mul(      c1,    NxNy, out=NN_tmp1 )
            alg.mul(      c3,    NyNx, out=NN_tmp2 )
            alg.sum( NN_tmp1, NN_tmp2, out=NN_tmp3 )
            alg.gauss_integrate( NN_tmp3, elem.dV, out = tmp12 ) # vx,uy
            
            # alg.gauss_integrate( c1*NyNx + c3*NxNy , elem.dV,out=tmp21) # vy,ux
            alg.mul(      c1,    NyNx, out=NN_tmp1 )
            alg.mul(      c3,    NxNy, out=NN_tmp2 )
            alg.sum( NN_tmp1, NN_tmp2, out=NN_tmp3 )
            alg.gauss_integrate( NN_tmp3, elem.dV, out = tmp21 ) # vy,ux
            
            # alg.gauss_integrate( c4*NyNy + c3*NxNx , elem.dV,out=tmp22) # vy,uy
            alg.mul(      c4,    NyNy, out=NN_tmp1 )
            alg.mul(      c3,    NxNx, out=NN_tmp2 )
            alg.sum( NN_tmp1, NN_tmp2, out=NN_tmp3 )
            alg.gauss_integrate( NN_tmp3, elem.dV, out = tmp22 ) # vy,uy
            
            
            
#             # alg.gauss_integrate( c4*NxNx + c3*NyNy + c3*NzNz , elem.dV,out=tmp11) # vx,ux
#             alg.mul(      c4,    NxNx, out=NN_tmp1 )
#             alg.mul(      c3,    NyNy, out=NN_tmp2 )
#             alg.mul(      c3,    NzNz, out=NN_tmp3 )
#             alg.sum( NN_tmp1, NN_tmp2, out=NN_tmp4 )
#             alg.sum( NN_tmp3, NN_tmp4, out=NN_tmp5 )
#             alg.gauss_integrate( NN_tmp5, elem.dV, out = tmp11 ) # vx,ux
            
#             # alg.gauss_integrate( c1*NxNy + c3*NyNx , elem.dV,out=tmp12) # vx,uy
#             alg.mul(      c1,    NxNy, out=NN_tmp1 )
#             alg.mul(      c3,    NyNx, out=NN_tmp2 )
#             alg.sum( NN_tmp1, NN_tmp2, out=NN_tmp3 )
#             alg.gauss_integrate( NN_tmp3, elem.dV, out = tmp12 ) # vx,uy
            
#             # alg.gauss_integrate( c1*NxNy + c3*NyNx , elem.dV,out=tmp12) # vx,uz
#             alg.mul(      c1,    NxNz, out=NN_tmp1 )
#             alg.mul(      c3,    NzNx, out=NN_tmp2 )
#             alg.sum( NN_tmp1, NN_tmp2, out=NN_tmp3 )
#             alg.gauss_integrate( NN_tmp3, elem.dV, out = tmp13 ) # vx,uz
            
            
            
            
#             # alg.gauss_integrate( c1*NyNx + c3*NxNy , elem.dV,out=tmp21) # vy,ux
#             alg.mul(      c1,    NyNx, out=NN_tmp1 )
#             alg.mul(      c3,    NxNy, out=NN_tmp2 )
#             alg.sum( NN_tmp1, NN_tmp2, out=NN_tmp3 )
#             alg.gauss_integrate( NN_tmp3, elem.dV, out = tmp21 ) # vy,ux
            
#             # alg.gauss_integrate( c4*NyNy + c3*NxNx +c3*NzNz, elem.dV,out=tmp22) # vy,uy
#             alg.mul(      c4,    NyNy, out=NN_tmp1 )
#             alg.mul(      c3,    NxNx, out=NN_tmp2 )
#             alg.mul(      c3,    NzNz, out=NN_tmp3 )
#             alg.sum( NN_tmp1, NN_tmp2, out=NN_tmp4 )
#             alg.sum( NN_tmp3, NN_tmp4, out=NN_tmp5 )
#             alg.gauss_integrate( NN_tmp5, elem.dV, out = tmp22 ) # vy,uy
            
#             # alg.gauss_integrate( c1*NyNx + c3*NxNy , elem.dV,out=tmp21) # vy,uz
#             alg.mul(      c1,    NyNz, out=NN_tmp1 )
#             alg.mul(      c3,    NzNy, out=NN_tmp2 )
#             alg.sum( NN_tmp1, NN_tmp2, out=NN_tmp3 )
#             alg.gauss_integrate( NN_tmp3, elem.dV, out = tmp23 ) # vy,uz
            
            
            

            
#             # alg.gauss_integrate( c1*NyNx + c3*NxNy , elem.dV,out=tmp21) # vz,ux
#             alg.mul(      c1,    NzNx, out=NN_tmp1 )
#             alg.mul(      c3,    NxNz, out=NN_tmp2 )
#             alg.sum( NN_tmp1, NN_tmp2, out=NN_tmp3 )
#             alg.gauss_integrate( NN_tmp3, elem.dV, out = tmp31 ) # vz,ux
            
#             # alg.gauss_integrate( c1*NyNx + c3*NxNy , elem.dV,out=tmp21) # vz,uy
#             alg.mul(      c1,    NzNy, out=NN_tmp1 )
#             alg.mul(      c3,    NyNz, out=NN_tmp2 )
#             alg.sum( NN_tmp1, NN_tmp2, out=NN_tmp3 )
#             alg.gauss_integrate( NN_tmp3, elem.dV, out = tmp32 ) # vz,uy
            
#             # alg.gauss_integrate( c4*NzNz + c3*NxNx +c3*NyNy, elem.dV,out=tmp33) # vz,uz
#             alg.mul(      c4,    NzNz, out=NN_tmp1 )
#             alg.mul(      c3,    NyNy, out=NN_tmp2 )
#             alg.mul(      c3,    NxNx, out=NN_tmp3 )
#             alg.sum( NN_tmp1, NN_tmp2, out=NN_tmp4 )
#             alg.sum( NN_tmp3, NN_tmp4, out=NN_tmp5 )
#             alg.gauss_integrate( NN_tmp5, elem.dV, out = tmp33 ) # vz,uz

                      
            # assemble globals
            
            for k in range(elems.size):
        
                ii=int(elems[k])
                
                i1 = ii
                i2 = ii +   nNodes # Move DOF for second variable
                i3 = ii + 2*nNodes # Move DOF for third variable
                
                for l in range(elems.size):

                    jj=int(elems[l])
                    
                    j1 = jj
                    j2 = jj +   nNodes # Move DOF for second variable
                    j3 = jj + 2*nNodes # Move DOF for second variable
                    
                    K[i1,j1] = K[i1,j1] + tmp11[k,l]                    
                    K[i1,j2] = K[i1,j2] + tmp12[k,l]                    
                    K[i1,j3] = K[i1,j3] + tmp13[k,l]
                    
                    K[i2,j1] = K[i2,j1] + tmp21[k,l]                    
                    K[i2,j2] = K[i2,j2] + tmp22[k,l]                    
                    K[i2,j3] = K[i2,j3] + tmp23[k,l]
                    
                    K[i3,j1] = K[i3,j1] + tmp31[k,l]                    
                    K[i3,j2] = K[i3,j2] + tmp32[k,l]                    
                    K[i3,j3] = K[i3,j3] + tmp33[k,l] +1 # Force zero

                # end for 

            # end for 
 
        # end for

    # end for

    fem.end_elements()   

    end_assmbly_time = time()
    
    
    # 
    els = Th.group_names['ri_face']['members'][0]
    for j in range(els['types'].size):

        elem = fem.element[ els['types'][j] ]

        if not elem.is_allocated():

            elem.end()
            elem.allocate(intorder=2)
            elem.allocate_spatial(ndim_analysis,compute_Jinv = False)

            nxel = alg.zeros( ( elem.nbasis, 1 ) )
            xel  = alg.zeros( ( elem.nbasis, 1 ) )
            nyel = alg.zeros( ( elem.nbasis, 1 ) )
            yel  = alg.zeros( ( elem.nbasis, 1 ) )
            

        # end if 

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):
            
            elems = elm_nodes[i,:]
            
            elem.set_coordinates(Th.x,Th.y,Th.z,elems)
            elem.compute_jacobian()
            
            N  = alg.transpose(elem.N)  
            
            for k in range(elems.size):
                xel[k,0] = Th.x[int(elems[k]),0]
                yel[k,0] = Th.y[int(elems[k]),0]
            # end for 
            
            nxel = xel/ri
            nyel = yel/ri
            
            
            tmp1 = alg.gauss_integrate( (Pi)*alg.dot_product(nxel,N)*N, elem.dV) # vx,ux
            tmp2 = alg.gauss_integrate( (Pi)*alg.dot_product(nyel,N)*N, elem.dV) # vy,ux
            
            # assemble globals            
            for k in range(elems.size):
        
                ii=int(elems[k])

                i1 = ii
                i2 = ii + nNodes
                i3 = ii + 2*nNodes
                
                f[i1,0] = f[i1,0] + tmp1[k,0]
                f[i2,0] = f[i2,0] + tmp2[k,0]

                # end for 

            # end for 
 
        # end for

    # end for

    fem.end_elements()  
    
    els = Th.group_names['ro_face']['members'][0]
    for j in range(els['types'].size):

        elem = fem.element[ els['types'][j] ]

        if not elem.is_allocated():

            elem.end()
            elem.allocate(intorder=2)
            elem.allocate_spatial(ndim_analysis,compute_Jinv = False)

            fh = alg.zeros( ( elem.nbasis, 1 ) )

        # end if 

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):
            
            elems = elm_nodes[i,:]
            
            elem.set_coordinates(Th.x,Th.y,Th.z,elems)
            elem.compute_jacobian()
            
            N  = alg.transpose(elem.N)
            
            for k in range(elems.size):
                xel[k,0] = Th.x[int(elems[k]),0]
                yel[k,0] = Th.y[int(elems[k]),0]
            # end for 
            
            nxel = xel/ro
            nyel = yel/ro
            
            
            tmp1 = alg.gauss_integrate( (-Po)*alg.dot_product(nxel,N)*N, elem.dV) # vx,ux
            tmp2 = alg.gauss_integrate( (-Po)*alg.dot_product(nyel,N)*N, elem.dV) # vy,ux
            

            # assemble globals            
            for k in range(elems.size):
        
                ii=int(elems[k])

                i1 = ii
                i2 = ii + nNodes
                i3 = ii + 2*nNodes
                
                f[i1,0] = f[i1,0] + tmp1[k,0]
                f[i2,0] = f[i2,0] + tmp2[k,0]

                # end for 

            # end for 
 
        # end for

    # end for

    fem.end_elements()
    
    # Setting Dirichlet BCs using TGV.
    TGV = 1e30
    
    # Every 1D node has dirichlet bc = 0
    els = Th.group_names['right_face']['members'][0]
    for j in range(els['types'].size):

        elm_nodes = np.unique(els['indices'][j])

        for ii_ in elm_nodes:
            i1 = int(ii_)          # ux
            i2 = int(ii_)+nNodes   # uy
            i3 = int(ii_)+2*nNodes # uz

            K[i2,i2] = TGV
            f[i2,0] = 0.0        

        # end for         
    # end for
    
    els = Th.group_names['left_face']['members'][0]
    for j in range(els['types'].size):

        elm_nodes = np.unique(els['indices'][j])

        for ii_ in elm_nodes:
            i1 = int(ii_)          # ux
            i2 = int(ii_)+nNodes   # uy
            i3 = int(ii_)+2*nNodes # uz

            K[i1,i1] = TGV
            f[i1,0] = 0.0
        # end for 
        
    # end for
    
    els = Th.group_names['top_face']['members'][0]
    for j in range(els['types'].size):

        elm_nodes = np.unique(els['indices'][j])

        for ii_ in elm_nodes:
            i1 = int(ii_)          # ux
            i2 = int(ii_)+nNodes   # uy
            i3 = int(ii_)+2*nNodes # uz

            K[i3,i3] = TGV
            f[i3,0]  = 0.0
        # end for 
        
    # end for
    els = Th.group_names['bottom_face']['members'][0]
    for j in range(els['types'].size):

        elm_nodes = np.unique(els['indices'][j])

        for ii_ in elm_nodes:
            i1 = int(ii_)          # ux
            i2 = int(ii_)+nNodes   # uy
            i3 = int(ii_)+2*nNodes # uz

            K[i3,i3] = TGV
            f[i3,0]  = 0.0
        # end for 
        
    # end for
    
    
    end_bc_time = time()
    
   
    u = alg.solve(K.tocsr(),f)
    # u = alg.solve_sparse_tests(K.tocsr(),f)
    
    end_solve_time = time()
    
    if stats:
        times['assembly'].append(  end_assmbly_time - start_time       )
        times['bc'].append( end_bc_time      - end_assmbly_time  )
        times['solve'].append(  end_solve_time   - end_bc_time      )
        times['total'].append( end_solve_time   - start_time        )
        
#         print("Assembly time:  {0:.6f} s ".format( end_assmbly_time - start_time       ) )
#         print("Boundary time:  {0:.6f} s ".format( end_bc_time      - end_assmbly_time ) )
#         print("Solution time:  {0:.6f} s ".format( end_solve_time   - end_bc_time      ) )
        print("Total run time: {0:.6f} s ".format( end_solve_time   - start_time       ) )
        print()
    # end if 

    return u,K,f

#-----------------------------------------------------------------------------------------------------

In [4]:
# r, dual, md2, om1n2, om1n10, oti
fem.set_global_algebra(dual)
alg = fem.get_global_algebra()





start_time = time()


Th = cylinder(1,2,1,he=0.1,quads=True,save=False, structured=True)
# Th = cylinder(1,2,he=0.0037,quads=False,save=True, structured=True)



end_time = time()

print(end_time - start_time,"seg")

# Find unused nodes.
unused_nodes = []
for i in range(Th.x.shape[0]):
    x = Th.x[i,0].real
    y = Th.y[i,0].real
    if x==0 and y ==0:
#         print(i)
        unused_nodes.append(i)
#

Th
        


0.6410772389999995 seg


/Users/maristi7/opt/anaconda3/envs/pyoti/lib/python3.7/site-packages/numpy/ctypeslib.py:523: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  return array(obj, copy=False)


< mesh (pyoti.static.onumm1n1) object with 2114 nodes, 2654 elements of types ( point1 (10), line2 (144), quad4 (850), hex8 (1650) ) >

In [5]:
times = {}
times['assembly'] = []
times['bc'] = []
times['solve'] = []
times['total'] = []

E  = alg.number(21e5)#+alg.e(1, order=10)#,order=1)#,1,10,2,1000
nu = alg.number(0.28)
ri = alg.number(1)
Pi = alg.number(10)# Pi = alg.number(1400)
ro = alg.number(2)
Po = alg.number(1000)

for i in range(1):
    u_res,K,f = solve_3d_linear_elasticity_prealloc(Th,E,nu,ri,Pi,ro,Po)
#end for

print('\n\nTimes:'+str(type(u_res)))
print("- Avg Assembly time:  {0:.6f} s ".format(np.average(times['assembly'] ) ) )
print("- Avg Boundary time:  {0:.6f} s ".format(np.average(times['bc'])      ) )
print("- Avg Solution time:  {0:.6f} s ".format(np.average(times['solve'])   ) )
print("- Avg Total run time: {0:.6f} s ".format(np.average(times['total'])   ) )

Total run time: 2.358872 s 



Times:<class 'pyoti.static.onumm1n1.omatm1n1'>
- Avg Assembly time:  1.046713 s 
- Avg Boundary time:  0.012220 s 
- Avg Solution time:  1.299939 s 
- Avg Total run time: 2.358872 s 


In [6]:
# Times:<class 'pyoti.static.onumm1n1.omatm1n1'>
# - Avg Assembly time:  16.480840 s 
# - Avg Boundary time:  0.076392 s 
# - Avg Solution time:  268.804248 s 
# - Avg Total run time: 285.361480 s 

In [7]:
u = alg.zeros((u_res.shape[0]//3,3))
u[:,0:1] = u_res[ :u_res.shape[0]//3, : ]
u[:,1:2] = u_res[   u_res.shape[0]//3:2*u_res.shape[0]//3, : ]
u[:,2:3] = u_res[ 2*u_res.shape[0]//3:, : ]

for i in unused_nodes:
    u[i,0:3] = np.nan
u.real.shape

(2114, 3)

In [33]:
Kcsr = K.tocsr()
Kcsr_r = Kcsr.real
Kcsr_r

<84246x84246 sparse matrix of type '<class 'numpy.float64'>'
	with 6376254 stored elements in Compressed Sparse Row format>

In [28]:
f_r = f.real

In [36]:
start_time = time()

lu = spla.splu(Kcsr_r.tocsc())
lu.solve(f_r)

end_time = time()
print("Real only system solve time: ",end_time - start_time," seg")

/Users/maristi7/opt/anaconda3/envs/pyoti/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:296: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


Real only system solve time:  239.94889127399983  seg


In [37]:
start_time = time()
lu.solve(f_r)
end_time = time()
print("New system solve time: ",end_time - start_time," seg")

New system solve time:  0.5369535199997699  seg


In [ ]:
# Real only system solve time:  235.89284812699952  seg (spsolve)

# Real only system solve time:  239.94889127399983  seg (superlu)
# New system solve time:  0.5369535199997699  seg (superlu)


In [8]:
import pyvista as pv
p = pv.BackgroundPlotter()
pv.set_plot_theme('default')
p.set_background("white")
grid = Th.to_pv(pd=[u.real],pd_names=['u'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [9]:
Th.group_names.keys()

dict_keys(['center', 'ri_left', 'ri_right', 'ro_left', 'ro_right', 'ri', 'ro', 'left', 'right', 'bottom_face', 'top_face', 'ri_face', 'right_face', 'ro_face', 'left_face', 'domain'])

In [10]:
Th.group_names['bottom_face']['members'][0]['indices'][0]

array([[   1,    9,  357,  108],
       [ 108,  357,  358,  107],
       [ 107,  358,  359,  106],
       ...,
       [1266,  130,  131, 1267],
       [1267,  131,  132, 1268],
       [1268,  132,    4,   47]], dtype=uint64)

In [5]:
f = alg.zeros(Th.x.shape)

for i in range(6):
    id_ = 301+i
    bound_name = Th.group_ids[id_]
    print(bound_name)
    for member in Th.group_names[bound_name]['members']:
        
        indices_list = member['indices']
        for indices in indices_list:

            for k in range(indices.shape[0]):
                for l in range(indices.shape[1]):
                    f[int(indices[k,l]),0]=i+1
                # end for 
            # end for 
        # end for 
    # end for 
# end for


Th.group_names['bottom_face']['members']

bottom_face
top_face
ri_face
right_face
ro_face
left_face


[{'types': array([3], dtype=int32),
  'tags': [array([ 56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,
           69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,
           82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,
           95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107,
          108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120,
          121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133,
          134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146,
          147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159,
          160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172,
          173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185,
          186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198,
          199, 200, 201, 202, 203, 204, 205], dtype=uint64)],
  'nodes': array([  2,   3,   4,   5,  10,  11,  12,  13,  14,  15,  

In [6]:
g = alg.zeros(Th.x.shape)
bound_name = 'right_face'
for member in Th.group_names[bound_name]['members']:

    indices_list = member['indices']
    for indices in indices_list:

        for k in range(indices.shape[0]):
            for l in range(indices.shape[1]):
                g[int(indices[k,l]),0]=3
            # end for 
        # end for 
    # end for 
# end for 

In [31]:
f.real

array([[0.],
       [6.],
       [4.],
       ...,
       [0.],
       [0.],
       [0.]])

In [8]:
import pyvista as pv
p = pv.BackgroundPlotter()
pv.set_plot_theme('default')
p.set_background("white")
grid = Th.to_pv(pd=[f.real],pd_names=['val'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=7,cmap='jet')
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [27]:
import pyvista as pv
p = pv.BackgroundPlotter()
pv.set_plot_theme('default')
p.set_background("white")
grid = Th.to_pv(pd=[g.real],pd_names=['val'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=7,cmap='jet')
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()